
<a id='whatsnew-0170'></a>

# v0.17.0 (October 9, 2015)

{{ header }}

This is a major release from 0.16.2 and includes a small number of API changes, several new features,
enhancements, and performance improvements along with a large number of bug fixes. We recommend that all
users upgrade to this version.

pandas >= 0.17.0 will no longer support compatibility with Python version 3.2 ([GH9118](https://github.com/pandas-dev/pandas/issues/9118))

The `pandas.io.data` package is deprecated and will be replaced by the
[pandas-datareader package](https://github.com/pydata/pandas-datareader).
This will allow the data modules to be independently updated to your pandas
installation. The API for `pandas-datareader v0.1.1` is exactly the same
as in `pandas v0.17.0` ([GH8961](https://github.com/pandas-dev/pandas/issues/8961), [GH10861](https://github.com/pandas-dev/pandas/issues/10861)).

After installing pandas-datareader, you can easily change your imports:

In [ ]:
from pandas.io import data, wb

becomes

In [ ]:
from pandas_datareader import data, wb

Highlights include:

- Release the Global Interpreter Lock (GIL) on some cython operations, see [here](#whatsnew-0170-gil)  
- Plotting methods are now available as attributes of the `.plot` accessor, see [here](#whatsnew-0170-plot)  
- The sorting API has been revamped to remove some long-time inconsistencies, see [here](#whatsnew-0170-api-breaking-sorting)  
- Support for a `datetime64[ns]` with timezones as a first-class dtype, see [here](#whatsnew-0170-tz)  
- The default for `to_datetime` will now be to `raise` when presented with unparseable formats,
  previously this would return the original input. Also, date parse
  functions now return consistent results. See [here](#whatsnew-0170-api-breaking-to-datetime)  
- The default for `dropna` in `HDFStore` has changed to `False`, to store by default all rows even
  if they are all `NaN`, see [here](#whatsnew-0170-api-breaking-hdf-dropna)  
- Datetime accessor (`dt`) now supports `Series.dt.strftime` to generate formatted strings for datetime-likes, and `Series.dt.total_seconds` to generate each duration of the timedelta in seconds. See [here](#whatsnew-0170-strftime)  
- `Period` and `PeriodIndex` can handle multiplied freq like `3D`, which corresponding to 3 days span. See [here](#whatsnew-0170-periodfreq)  
- Development installed versions of pandas will now have `PEP440` compliant version strings ([GH9518](https://github.com/pandas-dev/pandas/issues/9518))  
- Development support for benchmarking with the [Air Speed Velocity library](https://github.com/spacetelescope/asv/) ([GH8361](https://github.com/pandas-dev/pandas/issues/8361))  
- Support for reading SAS xport files, see [here](#whatsnew-0170-enhancements-sas-xport)  
- Documentation comparing SAS to *pandas*, see [here](getting_started/comparison/comparison_with_sas.ipynb#compare-with-sas)  
- Removal of the automatic TimeSeries broadcasting, deprecated since 0.8.0, see [here](#whatsnew-0170-prior-deprecations)  
- Display format with plain text can optionally align with Unicode East Asian Width, see [here](#whatsnew-0170-east-asian-width)  
- Compatibility with Python 3.5 ([GH11097](https://github.com/pandas-dev/pandas/issues/11097))  
- Compatibility with matplotlib 1.5.0 ([GH11111](https://github.com/pandas-dev/pandas/issues/11111))  


Check the [API Changes](#whatsnew-0170-api) and [deprecations](#whatsnew-0170-deprecations) before updating.

## What’s new in v0.17.0

- [New features](#New-features)  
  - [Datetime with TZ](#Datetime-with-TZ)  
  - [Releasing the GIL](#Releasing-the-GIL)  
  - [Plot submethods](#Plot-submethods)  
  - [Additional methods for `dt` accessor](#Additional-methods-for-`dt`-accessor)  
    - [strftime](#strftime)  
    - [total_seconds](#total_seconds)  
  - [Period frequency enhancement](#Period-frequency-enhancement)  
  - [Support for SAS XPORT files](#Support-for-SAS-XPORT-files)  
  - [Support for math functions in .eval()](#Support-for-math-functions-in-.eval())  
  - [Changes to Excel with `MultiIndex`](#Changes-to-Excel-with-`MultiIndex`)  
  - [Google BigQuery enhancements](#Google-BigQuery-enhancements)  
  - [Display alignment with Unicode East Asian width](#Display-alignment-with-Unicode-East-Asian-width)  
  - [Other enhancements](#Other-enhancements)  
- [Backwards incompatible API changes](#Backwards-incompatible-API-changes)  
  - [Changes to sorting API](#Changes-to-sorting-API)  
  - [Changes to to_datetime and to_timedelta](#Changes-to-to_datetime-and-to_timedelta)  
    - [Error handling](#Error-handling)  
    - [Consistent parsing](#Consistent-parsing)  
  - [Changes to Index comparisons](#Changes-to-Index-comparisons)  
  - [Changes to boolean comparisons vs. None](#Changes-to-boolean-comparisons-vs.-None)  
  - [HDFStore dropna behavior](#HDFStore-dropna-behavior)  
  - [Changes to `display.precision` option](#Changes-to-`display.precision`-option)  
  - [Changes to `Categorical.unique`](#Changes-to-`Categorical.unique`)  
  - [Changes to `bool` passed as `header` in parsers](#Changes-to-`bool`-passed-as-`header`-in-parsers)  
  - [Other API changes](#Other-API-changes)  
  - [Deprecations](#Deprecations)  
  - [Removal of prior version deprecations/changes](#Removal-of-prior-version-deprecations/changes)  
- [Performance improvements](#Performance-improvements)  
- [Bug fixes](#Bug-fixes)  
- [Contributors](#Contributors)  


<a id='whatsnew-0170-enhancements'></a>

## New features


<a id='whatsnew-0170-tz'></a>

### Datetime with TZ

We are adding an implementation that natively supports datetime with timezones. A `Series` or a `DataFrame` column previously
*could* be assigned a datetime with timezones, and would work as an `object` dtype. This had performance issues with a large
number rows. See the [docs](user_guide/timeseries.ipynb#timeseries-timezone-series) for more details. ([GH8260](https://github.com/pandas-dev/pandas/issues/8260), [GH10763](https://github.com/pandas-dev/pandas/issues/10763), [GH11034](https://github.com/pandas-dev/pandas/issues/11034)).

The new implementation allows for having a single-timezone across all rows, with operations in a performant manner.

This uses a new-dtype representation as well, that is very similar in look-and-feel to its numpy cousin `datetime64[ns]`

>**Note**
>
>There is a slightly different string repr for the underlying `DatetimeIndex` as a result of the dtype changes, but
functionally these are the same.

Previous behavior:

```ipython
In [1]: pd.date_range('20130101', periods=3, tz='US/Eastern')
Out[1]: DatetimeIndex(['2013-01-01 00:00:00-05:00', '2013-01-02 00:00:00-05:00',
                       '2013-01-03 00:00:00-05:00'],
                      dtype='datetime64[ns]', freq='D', tz='US/Eastern')

In [2]: pd.date_range('20130101', periods=3, tz='US/Eastern').dtype
Out[2]: dtype('<M8[ns]')
```


New behavior:


<a id='whatsnew-0170-gil'></a>

### Releasing the GIL

We are releasing the global-interpreter-lock (GIL) on some cython operations.
This will allow other threads to run simultaneously during computation, potentially allowing performance improvements
from multi-threading. Notably `groupby`, `nsmallest`, `value_counts` and some indexing operations benefit from this. ([GH8882](https://github.com/pandas-dev/pandas/issues/8882))

For example the groupby expression in the following code will have the GIL released during the factorization step, e.g. `df.groupby('key')`
as well as the `.sum()` operation.

In [ ]:
N = 1000000
ngroups = 10
df = DataFrame({'key': np.random.randint(0, ngroups, size=N),
                'data': np.random.randn(N)})
df.groupby('key')['data'].sum()

Releasing of the GIL could benefit an application that uses threads for user interactions (e.g. [QT](https://wiki.python.org/moin/PyQt)), or performing multi-threaded computations. A nice example of a library that can handle these types of computation-in-parallel is the [dask](https://dask.readthedocs.io/en/latest/) library.


<a id='whatsnew-0170-plot'></a>

### Plot submethods

The Series and DataFrame `.plot()` method allows for customizing [plot types](user_guide/visualization.ipynb#visualization-other) by supplying the `kind` keyword arguments. Unfortunately, many of these kinds of plots use different required and optional keyword arguments, which makes it difficult to discover what any given plot kind uses out of the dozens of possible arguments.

To alleviate this issue, we have added a new, optional plotting interface, which exposes each kind of plot as a method of the `.plot` attribute. Instead of writing `series.plot(kind=<kind>, ...)`, you can now also use `series.plot.<kind>(...)`:

<img src="whatsnew/../_static/whatsnew_plot_submethods.png" style="">

As a result of this change, these methods are now all discoverable via tab-completion:

Each method signature only includes relevant arguments. Currently, these are limited to required arguments, but in the future these will include optional arguments, as well. For an overview, see the new api.dataframe.plotting API documentation.


<a id='whatsnew-0170-strftime'></a>

### Additional methods for `dt` accessor

#### strftime

We are now supporting a `Series.dt.strftime` method for datetime-likes to generate a formatted string ([GH10110](https://github.com/pandas-dev/pandas/issues/10110)). Examples:

The string format is as the python standard library and details can be found [here](https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior)

#### total_seconds

`pd.Series` of type `timedelta64` has new method `.dt.total_seconds()` returning the duration of the timedelta in seconds ([GH10817](https://github.com/pandas-dev/pandas/issues/10817))


<a id='whatsnew-0170-periodfreq'></a>

### Period frequency enhancement

`Period`, `PeriodIndex` and `period_range` can now accept multiplied freq. Also, `Period.freq` and `PeriodIndex.freq` are now stored as a `DateOffset` instance like `DatetimeIndex`, and not as `str` ([GH7811](https://github.com/pandas-dev/pandas/issues/7811))

A multiplied freq represents a span of corresponding length. The example below creates a period of 3 days. Addition and subtraction will shift the period by its span.

You can use the multiplied freq in `PeriodIndex` and `period_range`.


<a id='whatsnew-0170-enhancements-sas-xport'></a>

### Support for SAS XPORT files

`read_sas()` provides support for reading *SAS XPORT* format files. ([GH4052](https://github.com/pandas-dev/pandas/issues/4052)).

In [ ]:
df = pd.read_sas('sas_xport.xpt')

It is also possible to obtain an iterator and read an XPORT file
incrementally.

In [ ]:
for df in pd.read_sas('sas_xport.xpt', chunksize=10000):
    do_something(df)

See the [docs](user_guide/io.ipynb#io-sas) for more details.


<a id='whatsnew-0170-matheval'></a>

### Support for math functions in .eval()

`eval()` now supports calling math functions ([GH4893](https://github.com/pandas-dev/pandas/issues/4893))

In [ ]:
df = pd.DataFrame({'a': np.random.randn(10)})
df.eval("b = sin(a)")

The support math functions are sin, cos, exp, log, expm1, log1p,
sqrt, sinh, cosh, tanh, arcsin, arccos, arctan, arccosh,
arcsinh, arctanh, abs and arctan2.

These functions map to the intrinsics for the `NumExpr` engine.  For the Python
engine, they are mapped to `NumPy` calls.

### Changes to Excel with `MultiIndex`

In version 0.16.2 a `DataFrame` with `MultiIndex` columns could not be written to Excel via `to_excel`.
That functionality has been added ([GH10564](https://github.com/pandas-dev/pandas/issues/10564)), along with updating  `read_excel` so that the data can
be read back with, no loss of information, by specifying which columns/rows make up the `MultiIndex`
in the `header` and `index_col` parameters ([GH4679](https://github.com/pandas-dev/pandas/issues/4679))

See the [documentation](user_guide/io.ipynb#io-excel) for more details.

Previously, it was necessary to specify the `has_index_names` argument in `read_excel`,
if the serialized data had index names.  For version 0.17.0 the output format of `to_excel`
has been changed to make this keyword unnecessary - the change is shown below.

**Old**

<img src="whatsnew/../_static/old-excel-index.png" style="">

**New**

<img src="whatsnew/../_static/new-excel-index.png" style="">

Excel files saved in version 0.16.2 or prior that had index names will still able to be read in,
but the `has_index_names` argument must specified to `True`.


<a id='whatsnew-0170-gbq'></a>

### Google BigQuery enhancements

- Added ability to automatically create a table/dataset using the `pandas.io.gbq.to_gbq()` function if the destination table/dataset does not exist. ([GH8325](https://github.com/pandas-dev/pandas/issues/8325), [GH11121](https://github.com/pandas-dev/pandas/issues/11121)).  
- Added ability to replace an existing table and schema when calling the `pandas.io.gbq.to_gbq()` function via the `if_exists` argument. See the [docs](https://pandas-gbq.readthedocs.io/en/latest/writing.html) for more details ([GH8325](https://github.com/pandas-dev/pandas/issues/8325)).  
- `InvalidColumnOrder` and `InvalidPageToken` in the gbq module will raise `ValueError` instead of `IOError`.  
- The `generate_bq_schema()` function is now deprecated and will be removed in a future version ([GH11121](https://github.com/pandas-dev/pandas/issues/11121))  
- The gbq module will now support Python 3 ([GH11094](https://github.com/pandas-dev/pandas/issues/11094)).  



<a id='whatsnew-0170-east-asian-width'></a>

### Display alignment with Unicode East Asian width

Enabling this option will affect the performance for printing of `DataFrame` and `Series` (about 2 times slower).
Use only when it is actually required.

Some East Asian countries use Unicode characters its width is corresponding to 2 alphabets. If a `DataFrame` or `Series` contains these characters, the default output cannot be aligned properly. The following options are added to enable precise handling for these characters.

- `display.unicode.east_asian_width`: Whether to use the Unicode East Asian Width to calculate the display text width. ([GH2612](https://github.com/pandas-dev/pandas/issues/2612))  
- `display.unicode.ambiguous_as_wide`: Whether to handle Unicode characters belong to Ambiguous as Wide. ([GH11102](https://github.com/pandas-dev/pandas/issues/11102))  


<img src="whatsnew/../_static/option_unicode01.png" style="">

<img src="whatsnew/../_static/option_unicode02.png" style="">

For further details, see [here](user_guide/options.ipynb#options-east-asian-width)


<a id='whatsnew-0170-enhancements-other'></a>

### Other enhancements

- Support for `openpyxl` >= 2.2. The API for style support is now stable ([GH10125](https://github.com/pandas-dev/pandas/issues/10125))  
- `merge` now accepts the argument `indicator` which adds a Categorical-type column (by default called `_merge`) to the output object that takes on the values ([GH8790](https://github.com/pandas-dev/pandas/issues/8790))  
    
  ``  
  ``  
  ``  
  ``  
  ``  
    
  ``  
  |Observation Origin|_merge value|
  |:---------------------------------:|:--------------:|
  |Merge key only in 'left' frame|left_only|
  |Merge key only in 'right' frame|right_only|
  |Merge key in both frames|both|
  For more, see the [updated docs](user_guide/merging.ipynb#merging-indicator)  
- `pd.to_numeric` is a new function to coerce strings to numbers (possibly with coercion) ([GH11133](https://github.com/pandas-dev/pandas/issues/11133))  
- `pd.merge` will now allow duplicate column names if they are not merged upon ([GH10639](https://github.com/pandas-dev/pandas/issues/10639)).  
- `pd.pivot` will now allow passing index as `None` ([GH3962](https://github.com/pandas-dev/pandas/issues/3962)).  
- `pd.concat` will now use existing Series names if provided ([GH10698](https://github.com/pandas-dev/pandas/issues/10698)).  
Previous behavior:  

```ipython
In [1]: pd.concat([foo, bar, baz], 1)
Out[1]:
      0  1  2
   0  1  1  4
   1  2  2  5
```


New behavior:  
- `DataFrame` has gained the `nlargest` and `nsmallest` methods ([GH10393](https://github.com/pandas-dev/pandas/issues/10393))  
- Add a `limit_direction` keyword argument that works with `limit` to enable `interpolate` to fill `NaN` values forward, backward, or both ([GH9218](https://github.com/pandas-dev/pandas/issues/9218), [GH10420](https://github.com/pandas-dev/pandas/issues/10420), [GH11115](https://github.com/pandas-dev/pandas/issues/11115))  
- Added a `DataFrame.round` method to round the values to a variable number of decimal places ([GH10568](https://github.com/pandas-dev/pandas/issues/10568)).  
- `drop_duplicates` and `duplicated` now accept a `keep` keyword to target first, last, and all duplicates. The `take_last` keyword is deprecated, see [here](#whatsnew-0170-deprecations) ([GH6511](https://github.com/pandas-dev/pandas/issues/6511), [GH8505](https://github.com/pandas-dev/pandas/issues/8505))  
- Reindex now has a `tolerance` argument that allows for finer control of [Limits on filling while reindexing](getting_started/basics.ipynb#basics-limits-on-reindex-fill) ([GH10411](https://github.com/pandas-dev/pandas/issues/10411)):  
  When used on a `DatetimeIndex`, `TimedeltaIndex` or `PeriodIndex`, `tolerance` will coerced into a `Timedelta` if possible. This allows you to specify tolerance with a string:  
  `tolerance` is also exposed by the lower level `Index.get_indexer` and `Index.get_loc` methods.  
- Added functionality to use the `base` argument when resampling a `TimeDeltaIndex` ([GH10530](https://github.com/pandas-dev/pandas/issues/10530))  
- `DatetimeIndex` can be instantiated using strings contains `NaT` ([GH7599](https://github.com/pandas-dev/pandas/issues/7599))  
- `to_datetime` can now accept the `yearfirst` keyword ([GH7599](https://github.com/pandas-dev/pandas/issues/7599))  
- `pandas.tseries.offsets` larger than the `Day` offset can now be used with a `Series` for addition/subtraction ([GH10699](https://github.com/pandas-dev/pandas/issues/10699)).  See the [docs](user_guide/timeseries.ipynb#timeseries-offsetseries) for more details.  
- `pd.Timedelta.total_seconds()` now returns Timedelta duration to ns precision (previously microsecond precision) ([GH10939](https://github.com/pandas-dev/pandas/issues/10939))  
- `PeriodIndex` now supports arithmetic with `np.ndarray` ([GH10638](https://github.com/pandas-dev/pandas/issues/10638))  
- Support pickling of `Period` objects ([GH10439](https://github.com/pandas-dev/pandas/issues/10439))  
- `.as_blocks` will now take a `copy` optional argument to return a copy of the data, default is to copy (no change in behavior from prior versions), ([GH9607](https://github.com/pandas-dev/pandas/issues/9607))  
- `regex` argument to `DataFrame.filter` now handles numeric column names instead of raising `ValueError` ([GH10384](https://github.com/pandas-dev/pandas/issues/10384)).  
- Enable reading gzip compressed files via URL, either by explicitly setting the compression parameter or by inferring from the presence of the HTTP Content-Encoding header in the response ([GH8685](https://github.com/pandas-dev/pandas/issues/8685))  
- Enable writing Excel files in [memory](user_guide/io.ipynb#io-excel-writing-buffer) using StringIO/BytesIO ([GH7074](https://github.com/pandas-dev/pandas/issues/7074))  
- Enable serialization of lists and dicts to strings in `ExcelWriter` ([GH8188](https://github.com/pandas-dev/pandas/issues/8188))  
- SQL io functions now accept a SQLAlchemy connectable. ([GH7877](https://github.com/pandas-dev/pandas/issues/7877))  
- `pd.read_sql` and `to_sql` can accept database URI as `con` parameter ([GH10214](https://github.com/pandas-dev/pandas/issues/10214))  
- `read_sql_table` will now allow reading from views ([GH10750](https://github.com/pandas-dev/pandas/issues/10750)).  
- Enable writing complex values to `HDFStores` when using the `table` format ([GH10447](https://github.com/pandas-dev/pandas/issues/10447))  
- Enable `pd.read_hdf` to be used without specifying a key when the HDF file contains a single dataset ([GH10443](https://github.com/pandas-dev/pandas/issues/10443))  
- `pd.read_stata` will now read Stata 118 type files. ([GH9882](https://github.com/pandas-dev/pandas/issues/9882))  
- `msgpack` submodule has been updated to 0.4.6 with backward compatibility ([GH10581](https://github.com/pandas-dev/pandas/issues/10581))  
- `DataFrame.to_dict` now accepts `orient='index'` keyword argument ([GH10844](https://github.com/pandas-dev/pandas/issues/10844)).  
- `DataFrame.apply` will return a Series of dicts if the passed function returns a dict and `reduce=True` ([GH8735](https://github.com/pandas-dev/pandas/issues/8735)).  
- Allow passing kwargs to the interpolation methods ([GH10378](https://github.com/pandas-dev/pandas/issues/10378)).  
- Improved error message when concatenating an empty iterable of `Dataframe` objects ([GH9157](https://github.com/pandas-dev/pandas/issues/9157))  
- `pd.read_csv` can now read bz2-compressed files incrementally, and the C parser can read bz2-compressed files from AWS S3 ([GH11070](https://github.com/pandas-dev/pandas/issues/11070), [GH11072](https://github.com/pandas-dev/pandas/issues/11072)).  
- In `pd.read_csv`, recognize `s3n://` and `s3a://` URLs as designating S3 file storage ([GH11070](https://github.com/pandas-dev/pandas/issues/11070), [GH11071](https://github.com/pandas-dev/pandas/issues/11071)).  
- Read CSV files from AWS S3 incrementally, instead of first downloading the entire file. (Full file download still required for compressed files in Python 2.)  ([GH11070](https://github.com/pandas-dev/pandas/issues/11070), [GH11073](https://github.com/pandas-dev/pandas/issues/11073))  
- `pd.read_csv` is now able to infer compression type for files read from AWS S3 storage ([GH11070](https://github.com/pandas-dev/pandas/issues/11070), [GH11074](https://github.com/pandas-dev/pandas/issues/11074)).  



<a id='whatsnew-0170-api'></a>

<a id='whatsnew-0170-api-breaking'></a>

## Backwards incompatible API changes


<a id='whatsnew-0170-api-breaking-sorting'></a>

### Changes to sorting API

The sorting API has had some longtime inconsistencies. ([GH9816](https://github.com/pandas-dev/pandas/issues/9816), [GH8239](https://github.com/pandas-dev/pandas/issues/8239)).

Here is a summary of the API **PRIOR** to 0.17.0:

- `Series.sort` is **INPLACE** while `DataFrame.sort` returns a new object.  
- `Series.order` returns a new object  
- It was possible to use `Series/DataFrame.sort_index` to sort by **values** by passing the `by` keyword.  
- `Series/DataFrame.sortlevel` worked only on a `MultiIndex` for sorting by index.  


To address these issues, we have revamped the API:

- We have introduced a new method, `DataFrame.sort_values()`, which is the merger of `DataFrame.sort()`, `Series.sort()`,
  and `Series.order()`, to handle sorting of **values**.  
- The existing methods `Series.sort()`, `Series.order()`, and `DataFrame.sort()` have been deprecated and will be removed in a
  future version.  
- The `by` argument of `DataFrame.sort_index()` has been deprecated and will be removed in a future version.  
- The existing method `.sort_index()` will gain the `level` keyword to enable level sorting.  


We now have two distinct and non-overlapping methods of sorting. A `*` marks items that
will show a `FutureWarning`.

To sort by the **values**:

````````````|Previous|Replacement|
|:--------------------------------:|:----------------------------------:|
|* Series.order()|Series.sort_values()|
|* Series.sort()|Series.sort_values(inplace=True)|
|* DataFrame.sort(columns=...)|DataFrame.sort_values(by=...)|
To sort by the **index**:

````````````````````|Previous|Replacement|
|:--------------------------------:|:----------------------------------:|
|Series.sort_index()|Series.sort_index()|
|Series.sortlevel(level=...)|Series.sort_index(level=...)|
|DataFrame.sort_index()|DataFrame.sort_index()|
|DataFrame.sortlevel(level=...)|DataFrame.sort_index(level=...)|
|* DataFrame.sort()|DataFrame.sort_index()|
We have also deprecated and changed similar methods in two Series-like classes, `Index` and `Categorical`.

````````|Previous|Replacement|
|:--------------------------------:|:----------------------------------:|
|* Index.order()|Index.sort_values()|
|* Categorical.order()|Categorical.sort_values()|

<a id='whatsnew-0170-api-breaking-to-datetime'></a>

### Changes to to_datetime and to_timedelta

#### Error handling

The default for `pd.to_datetime` error handling has changed to `errors='raise'`.
In prior versions it was `errors='ignore'`. Furthermore, the `coerce` argument
has been deprecated in favor of `errors='coerce'`. This means that invalid parsing
will raise rather that return the original input as in previous versions. ([GH10636](https://github.com/pandas-dev/pandas/issues/10636))

Previous behavior:

```ipython
In [2]: pd.to_datetime(['2009-07-31', 'asd'])
Out[2]: array(['2009-07-31', 'asd'], dtype=object)
```


New behavior:

```ipython
In [3]: pd.to_datetime(['2009-07-31', 'asd'])
ValueError: Unknown string format
```


Of course you can coerce this as well.

To keep the previous behavior, you can use `errors='ignore'`:

Furthermore, `pd.to_timedelta` has gained a similar API, of `errors='raise'|'ignore'|'coerce'`, and the `coerce` keyword
has been deprecated in favor of `errors='coerce'`.

#### Consistent parsing

The string parsing of `to_datetime`, `Timestamp` and `DatetimeIndex` has
been made consistent. ([GH7599](https://github.com/pandas-dev/pandas/issues/7599))

Prior to v0.17.0, `Timestamp` and `to_datetime` may parse year-only datetime-string incorrectly using today’s date, otherwise `DatetimeIndex`
uses the beginning of the year. `Timestamp` and `to_datetime` may raise `ValueError` in some types of datetime-string which `DatetimeIndex`
can parse, such as a quarterly string.

Previous behavior:

```ipython
In [1]: pd.Timestamp('2012Q2')
Traceback
   ...
ValueError: Unable to parse 2012Q2

# Results in today's date.
In [2]: pd.Timestamp('2014')
Out [2]: 2014-08-12 00:00:00
```


v0.17.0 can parse them as below. It works on `DatetimeIndex` also.

New behavior:

>**Note**
>
>If you want to perform calculations based on today’s date, use `Timestamp.now()` and `pandas.tseries.offsets`.

### Changes to Index comparisons

Operator equal on `Index` should behavior similarly to `Series` ([GH9947](https://github.com/pandas-dev/pandas/issues/9947), [GH10637](https://github.com/pandas-dev/pandas/issues/10637))

Starting in v0.17.0, comparing `Index` objects of different lengths will raise
a `ValueError`. This is to be consistent with the behavior of `Series`.

Previous behavior:

```ipython
In [2]: pd.Index([1, 2, 3]) == pd.Index([1, 4, 5])
Out[2]: array([ True, False, False], dtype=bool)

In [3]: pd.Index([1, 2, 3]) == pd.Index([2])
Out[3]: array([False,  True, False], dtype=bool)

In [4]: pd.Index([1, 2, 3]) == pd.Index([1, 2])
Out[4]: False
```


New behavior:

```ipython
In [8]: pd.Index([1, 2, 3]) == pd.Index([1, 4, 5])
Out[8]: array([ True, False, False], dtype=bool)

In [9]: pd.Index([1, 2, 3]) == pd.Index([2])
ValueError: Lengths must match to compare

In [10]: pd.Index([1, 2, 3]) == pd.Index([1, 2])
ValueError: Lengths must match to compare
```


Note that this is different from the `numpy` behavior where a comparison can
be broadcast:

or it can return False if broadcasting can not be done:

### Changes to boolean comparisons vs. None

Boolean comparisons of a `Series` vs `None` will now be equivalent to comparing with `np.nan`, rather than raise `TypeError`. ([GH1079](https://github.com/pandas-dev/pandas/issues/1079)).

Previous behavior:

```ipython
In [5]: s == None
TypeError: Could not compare <type 'NoneType'> type with Series
```


New behavior:

Usually you simply want to know which values are null.

You generally will want to use `isnull/notnull` for these types of comparisons, as `isnull/notnull` tells you which elements are null. One has to be
mindful that `nan's` don’t compare equal, but `None's` do. Note that Pandas/numpy uses the fact that `np.nan != np.nan`, and treats `None` like `np.nan`.


<a id='whatsnew-0170-api-breaking-hdf-dropna'></a>

### HDFStore dropna behavior

The default behavior for HDFStore write functions with `format='table'` is now to keep rows that are all missing. Previously, the behavior was to drop rows that were all missing save the index. The previous behavior can be replicated using the `dropna=True` option. ([GH9382](https://github.com/pandas-dev/pandas/issues/9382))

Previous behavior:

```ipython
In [27]:
df_with_missing.to_hdf('file.h5',
                       'df_with_missing',
                       format='table',
                       mode='w')

In [28]: pd.read_hdf('file.h5', 'df_with_missing')

Out [28]:
      col1  col2
  0     0     1
  2     2   NaN
```


New behavior:

See the [docs](user_guide/io.ipynb#io-hdf5) for more details.


<a id='whatsnew-0170-api-breaking-display-precision'></a>

### Changes to `display.precision` option

The `display.precision` option has been clarified to refer to decimal places ([GH10451](https://github.com/pandas-dev/pandas/issues/10451)).

Earlier versions of pandas would format floating point numbers to have one less decimal place than the value in
`display.precision`.

```ipython
In [1]: pd.set_option('display.precision', 2)

In [2]: pd.DataFrame({'x': [123.456789]})
Out[2]:
       x
0  123.5
```


If interpreting precision as “significant figures” this did work for scientific notation but that same interpretation
did not work for values with standard formatting. It was also out of step with how numpy handles formatting.

Going forward the value of `display.precision` will directly control the number of places after the decimal, for
regular formatting as well as scientific notation, similar to how numpy’s `precision` print option works.

To preserve output behavior with prior versions the default value of `display.precision` has been reduced to `6`
from `7`.


<a id='whatsnew-0170-api-breaking-categorical-unique'></a>

### Changes to `Categorical.unique`

`Categorical.unique` now returns new `Categoricals` with `categories` and `codes` that are unique, rather than returning `np.array` ([GH10508](https://github.com/pandas-dev/pandas/issues/10508))

- unordered category: values and categories are sorted by appearance order.  
- ordered category: values are sorted by appearance order, categories keep existing order.  

### Changes to `bool` passed as `header` in parsers

In earlier versions of pandas, if a bool was passed the `header` argument of
`read_csv`, `read_excel`, or `read_html` it was implicitly converted to
an integer, resulting in `header=0` for `False` and `header=1` for `True`
([GH6113](https://github.com/pandas-dev/pandas/issues/6113))

A `bool` input to `header` will now raise a `TypeError`

```ipython
In [29]: df = pd.read_csv('data.csv', header=False)
TypeError: Passing a bool to header is invalid. Use header=None for no header or
header=int or list-like of ints to specify the row(s) making up the column names
```



<a id='whatsnew-0170-api-breaking-other'></a>

### Other API changes

- Line and kde plot with `subplots=True` now uses default colors, not all black. Specify `color='k'` to draw all lines in black ([GH9894](https://github.com/pandas-dev/pandas/issues/9894))  
- Calling the `.value_counts()` method on a Series with a `categorical` dtype now returns a Series with a `CategoricalIndex` ([GH10704](https://github.com/pandas-dev/pandas/issues/10704))  
- The metadata properties of subclasses of pandas objects will now be serialized ([GH10553](https://github.com/pandas-dev/pandas/issues/10553)).  
- `groupby` using `Categorical` follows the same rule as `Categorical.unique` described above  ([GH10508](https://github.com/pandas-dev/pandas/issues/10508))  
- When constructing `DataFrame` with an array of `complex64` dtype previously meant the corresponding column
  was automatically promoted to the `complex128` dtype. Pandas will now preserve the itemsize of the input for complex data ([GH10952](https://github.com/pandas-dev/pandas/issues/10952))  
- some numeric reduction operators would return `ValueError`, rather than `TypeError` on object types that includes strings and numbers ([GH11131](https://github.com/pandas-dev/pandas/issues/11131))  
- Passing currently unsupported `chunksize` argument to `read_excel` or `ExcelFile.parse` will now raise `NotImplementedError` ([GH8011](https://github.com/pandas-dev/pandas/issues/8011))  
- Allow an `ExcelFile` object to be passed into `read_excel` ([GH11198](https://github.com/pandas-dev/pandas/issues/11198))  
- `DatetimeIndex.union` does not infer `freq` if `self` and the input have `None` as `freq` ([GH11086](https://github.com/pandas-dev/pandas/issues/11086))  
- `NaT`’s methods now either raise `ValueError`, or return `np.nan` or `NaT` ([GH9513](https://github.com/pandas-dev/pandas/issues/9513))  
    
    
  ``  
  ````  
  ``  
  ``````````  
  ``  
  ``  
  ``  
    
  |Behavior|Methods|
  |:-----------------------------:|:-------------------------------------------------------------:|
  |return np.nan|weekday, isoweekday|
  |return NaT|date, now, replace, to_datetime, today|
  |return np.datetime64('NaT')|to_datetime64 (unchanged)|
  |raise ValueError|All other public methods (names not beginning with underscores)|



<a id='whatsnew-0170-deprecations'></a>

### Deprecations

- For `Series` the following indexing functions are deprecated ([GH10177](https://github.com/pandas-dev/pandas/issues/10177)).  
    
    
  ``  
  ````  
  ``  
  ````  
  ``  
  ````  
  |Deprecated Function|Replacement|
  |:-------------------:|:-------------------------------:|
  |.irow(i)|.iloc[i] or .iat[i]|
  |.iget(i)|.iloc[i] or .iat[i]|
  |.iget_value(i)|.iloc[i] or .iat[i]|
- For `DataFrame` the following indexing functions are deprecated ([GH10177](https://github.com/pandas-dev/pandas/issues/10177)).  
    
    
  ``  
  ``  
  ``  
  ````  
  ``  
  ``  
  |Deprecated Function|Replacement|
  |:-------------------:|:-------------------------------:|
  |.irow(i)|.iloc[i]|
  |.iget_value(i, j)|.iloc[i, j] or .iat[i, j]|
  |.icol(j)|.iloc[:, j]|


>**Note**
>
>These indexing function have been deprecated in the documentation since 0.11.0.

- `Categorical.name` was deprecated to make `Categorical` more `numpy.ndarray` like. Use `Series(cat, name="whatever")` instead ([GH10482](https://github.com/pandas-dev/pandas/issues/10482)).  
- Setting missing values (NaN) in a `Categorical`’s `categories` will issue a warning ([GH10748](https://github.com/pandas-dev/pandas/issues/10748)). You can still have missing values in the `values`.  
- `drop_duplicates` and `duplicated`’s `take_last` keyword was deprecated in favor of `keep`. ([GH6511](https://github.com/pandas-dev/pandas/issues/6511), [GH8505](https://github.com/pandas-dev/pandas/issues/8505))  
- `Series.nsmallest` and `nlargest`’s `take_last` keyword was deprecated in favor of `keep`. ([GH10792](https://github.com/pandas-dev/pandas/issues/10792))  
- `DataFrame.combineAdd` and `DataFrame.combineMult` are deprecated. They
  can easily be replaced by using the `add` and `mul` methods:
  `DataFrame.add(other, fill_value=0)` and `DataFrame.mul(other, fill_value=1.)`
  ([GH10735](https://github.com/pandas-dev/pandas/issues/10735)).  
- `TimeSeries` deprecated in favor of `Series` (note that this has been an alias since 0.13.0), ([GH10890](https://github.com/pandas-dev/pandas/issues/10890))  
- `SparsePanel` deprecated and will be removed in a future version ([GH11157](https://github.com/pandas-dev/pandas/issues/11157)).  
- `Series.is_time_series` deprecated in favor of `Series.index.is_all_dates` ([GH11135](https://github.com/pandas-dev/pandas/issues/11135))  
- Legacy offsets (like `'A@JAN'`) are deprecated (note that this has been alias since 0.8.0) ([GH10878](https://github.com/pandas-dev/pandas/issues/10878))  
- `WidePanel` deprecated in favor of `Panel`, `LongPanel` in favor of `DataFrame` (note these have been aliases since < 0.11.0), ([GH10892](https://github.com/pandas-dev/pandas/issues/10892))  
- `DataFrame.convert_objects` has been deprecated in favor of type-specific functions `pd.to_datetime`, `pd.to_timestamp` and `pd.to_numeric` (new in 0.17.0) ([GH11133](https://github.com/pandas-dev/pandas/issues/11133)).  



<a id='whatsnew-0170-prior-deprecations'></a>

### Removal of prior version deprecations/changes

- Removal of `na_last` parameters from `Series.order()` and `Series.sort()`, in favor of `na_position`. ([GH5231](https://github.com/pandas-dev/pandas/issues/5231))  
- Remove of `percentile_width` from `.describe()`, in favor of `percentiles`. ([GH7088](https://github.com/pandas-dev/pandas/issues/7088))  
- Removal of `colSpace` parameter from `DataFrame.to_string()`, in favor of `col_space`, circa 0.8.0 version.  
- Removal of automatic time-series broadcasting ([GH2304](https://github.com/pandas-dev/pandas/issues/2304))  
Previously  

```ipython
In [3]: df + df.A
FutureWarning: TimeSeries broadcasting along DataFrame index by default is deprecated.
Please use DataFrame.<op> to explicitly broadcast arithmetic operations along the index

Out[3]:
                    A         B
2013-01-01  0.942870 -0.719541
2013-01-02  2.865414  1.120055
2013-01-03 -1.441177  0.166574
2013-01-04  1.719177  0.223065
2013-01-05  0.031393 -2.226989
```


Current  
- Remove `table` keyword in `HDFStore.put/append`, in favor of using `format=` ([GH4645](https://github.com/pandas-dev/pandas/issues/4645))  
- Remove `kind` in `read_excel/ExcelFile` as its unused ([GH4712](https://github.com/pandas-dev/pandas/issues/4712))  
- Remove `infer_type` keyword from `pd.read_html` as its unused ([GH4770](https://github.com/pandas-dev/pandas/issues/4770), [GH7032](https://github.com/pandas-dev/pandas/issues/7032))  
- Remove `offset` and `timeRule` keywords from `Series.tshift/shift`, in favor of `freq` ([GH4853](https://github.com/pandas-dev/pandas/issues/4853), [GH4864](https://github.com/pandas-dev/pandas/issues/4864))  
- Remove `pd.load/pd.save` aliases in favor of `pd.to_pickle/pd.read_pickle` ([GH3787](https://github.com/pandas-dev/pandas/issues/3787))  



<a id='whatsnew-0170-performance'></a>

## Performance improvements

- Development support for benchmarking with the [Air Speed Velocity library](https://github.com/spacetelescope/asv/) ([GH8361](https://github.com/pandas-dev/pandas/issues/8361))  
- Added vbench benchmarks for alternative ExcelWriter engines and reading Excel files ([GH7171](https://github.com/pandas-dev/pandas/issues/7171))  
- Performance improvements in `Categorical.value_counts` ([GH10804](https://github.com/pandas-dev/pandas/issues/10804))  
- Performance improvements in `SeriesGroupBy.nunique` and `SeriesGroupBy.value_counts` and `SeriesGroupby.transform` ([GH10820](https://github.com/pandas-dev/pandas/issues/10820), [GH11077](https://github.com/pandas-dev/pandas/issues/11077))  
- Performance improvements in `DataFrame.drop_duplicates` with integer dtypes ([GH10917](https://github.com/pandas-dev/pandas/issues/10917))  
- Performance improvements in `DataFrame.duplicated` with wide frames. ([GH10161](https://github.com/pandas-dev/pandas/issues/10161), [GH11180](https://github.com/pandas-dev/pandas/issues/11180))  
- 4x improvement in `timedelta` string parsing ([GH6755](https://github.com/pandas-dev/pandas/issues/6755), [GH10426](https://github.com/pandas-dev/pandas/issues/10426))  
- 8x improvement in `timedelta64` and `datetime64` ops ([GH6755](https://github.com/pandas-dev/pandas/issues/6755))  
- Significantly improved performance of indexing `MultiIndex` with slicers ([GH10287](https://github.com/pandas-dev/pandas/issues/10287))  
- 8x improvement in `iloc` using list-like input ([GH10791](https://github.com/pandas-dev/pandas/issues/10791))  
- Improved performance of `Series.isin` for datetimelike/integer Series ([GH10287](https://github.com/pandas-dev/pandas/issues/10287))  
- 20x improvement in `concat` of Categoricals when categories are identical ([GH10587](https://github.com/pandas-dev/pandas/issues/10587))  
- Improved performance of `to_datetime` when specified format string is ISO8601 ([GH10178](https://github.com/pandas-dev/pandas/issues/10178))  
- 2x improvement of `Series.value_counts` for float dtype ([GH10821](https://github.com/pandas-dev/pandas/issues/10821))  
- Enable `infer_datetime_format` in `to_datetime` when date components do not have 0 padding ([GH11142](https://github.com/pandas-dev/pandas/issues/11142))  
- Regression from 0.16.1 in constructing `DataFrame` from nested dictionary ([GH11084](https://github.com/pandas-dev/pandas/issues/11084))  
- Performance improvements in addition/subtraction operations for `DateOffset` with `Series` or `DatetimeIndex`  ([GH10744](https://github.com/pandas-dev/pandas/issues/10744), [GH11205](https://github.com/pandas-dev/pandas/issues/11205))  



<a id='whatsnew-0170-bug-fixes'></a>

## Bug fixes

- Bug in incorrect computation of `.mean()` on `timedelta64[ns]` because of overflow ([GH9442](https://github.com/pandas-dev/pandas/issues/9442))  
- Bug in  `.isin` on older numpies ([GH11232](https://github.com/pandas-dev/pandas/issues/11232))  
- Bug in `DataFrame.to_html(index=False)` renders unnecessary `name` row ([GH10344](https://github.com/pandas-dev/pandas/issues/10344))  
- Bug in `DataFrame.to_latex()` the `column_format` argument could not be passed ([GH9402](https://github.com/pandas-dev/pandas/issues/9402))  
- Bug in `DatetimeIndex` when localizing with `NaT` ([GH10477](https://github.com/pandas-dev/pandas/issues/10477))  
- Bug in `Series.dt` ops in preserving meta-data ([GH10477](https://github.com/pandas-dev/pandas/issues/10477))  
- Bug in preserving `NaT` when passed in an otherwise invalid `to_datetime` construction ([GH10477](https://github.com/pandas-dev/pandas/issues/10477))  
- Bug in `DataFrame.apply` when function returns categorical series. ([GH9573](https://github.com/pandas-dev/pandas/issues/9573))  
- Bug in `to_datetime` with invalid dates and formats supplied ([GH10154](https://github.com/pandas-dev/pandas/issues/10154))  
- Bug in `Index.drop_duplicates` dropping name(s) ([GH10115](https://github.com/pandas-dev/pandas/issues/10115))  
- Bug in `Series.quantile` dropping name ([GH10881](https://github.com/pandas-dev/pandas/issues/10881))  
- Bug in `pd.Series` when setting a value on an empty `Series` whose index has a frequency. ([GH10193](https://github.com/pandas-dev/pandas/issues/10193))  
- Bug in `pd.Series.interpolate` with invalid `order` keyword values. ([GH10633](https://github.com/pandas-dev/pandas/issues/10633))  
- Bug in `DataFrame.plot` raises `ValueError` when color name is specified by multiple characters ([GH10387](https://github.com/pandas-dev/pandas/issues/10387))  
- Bug in `Index` construction with a mixed list of tuples ([GH10697](https://github.com/pandas-dev/pandas/issues/10697))  
- Bug in `DataFrame.reset_index` when index contains `NaT`. ([GH10388](https://github.com/pandas-dev/pandas/issues/10388))  
- Bug in `ExcelReader` when worksheet is empty ([GH6403](https://github.com/pandas-dev/pandas/issues/6403))  
- Bug in `BinGrouper.group_info` where returned values are not compatible with base class ([GH10914](https://github.com/pandas-dev/pandas/issues/10914))  
- Bug in clearing the cache on `DataFrame.pop` and a subsequent inplace op ([GH10912](https://github.com/pandas-dev/pandas/issues/10912))  
- Bug in indexing with a mixed-integer `Index` causing an `ImportError` ([GH10610](https://github.com/pandas-dev/pandas/issues/10610))  
- Bug in `Series.count` when index has nulls ([GH10946](https://github.com/pandas-dev/pandas/issues/10946))  
- Bug in pickling of a non-regular freq `DatetimeIndex` ([GH11002](https://github.com/pandas-dev/pandas/issues/11002))  
- Bug causing `DataFrame.where` to not respect the `axis` parameter when the frame has a symmetric shape. ([GH9736](https://github.com/pandas-dev/pandas/issues/9736))  
- Bug in `Table.select_column` where name is not preserved ([GH10392](https://github.com/pandas-dev/pandas/issues/10392))  
- Bug in `offsets.generate_range` where `start` and `end` have finer precision than `offset` ([GH9907](https://github.com/pandas-dev/pandas/issues/9907))  
- Bug in `pd.rolling_*` where `Series.name` would be lost in the output ([GH10565](https://github.com/pandas-dev/pandas/issues/10565))  
- Bug in `stack` when index or columns are not unique. ([GH10417](https://github.com/pandas-dev/pandas/issues/10417))  
- Bug in setting a `Panel` when an axis has a MultiIndex ([GH10360](https://github.com/pandas-dev/pandas/issues/10360))  
- Bug in `USFederalHolidayCalendar` where `USMemorialDay` and `USMartinLutherKingJr` were incorrect ([GH10278](https://github.com/pandas-dev/pandas/issues/10278) and [GH9760](https://github.com/pandas-dev/pandas/issues/9760) )  
- Bug in `.sample()` where returned object, if set, gives unnecessary `SettingWithCopyWarning` ([GH10738](https://github.com/pandas-dev/pandas/issues/10738))  
- Bug in `.sample()` where weights passed as `Series` were not aligned along axis before being treated positionally, potentially causing problems if weight indices were not aligned with sampled object. ([GH10738](https://github.com/pandas-dev/pandas/issues/10738))  
- Regression fixed in ([GH9311](https://github.com/pandas-dev/pandas/issues/9311), [GH6620](https://github.com/pandas-dev/pandas/issues/6620), [GH9345](https://github.com/pandas-dev/pandas/issues/9345)), where groupby with a datetime-like converting to float with certain aggregators ([GH10979](https://github.com/pandas-dev/pandas/issues/10979))  
- Bug in `DataFrame.interpolate` with `axis=1` and `inplace=True` ([GH10395](https://github.com/pandas-dev/pandas/issues/10395))  
- Bug in `io.sql.get_schema` when specifying multiple columns as primary
  key ([GH10385](https://github.com/pandas-dev/pandas/issues/10385)).  
- Bug in `groupby(sort=False)` with datetime-like `Categorical` raises `ValueError` ([GH10505](https://github.com/pandas-dev/pandas/issues/10505))  
- Bug in `groupby(axis=1)` with `filter()` throws `IndexError` ([GH11041](https://github.com/pandas-dev/pandas/issues/11041))  
- Bug in `test_categorical` on big-endian builds ([GH10425](https://github.com/pandas-dev/pandas/issues/10425))  
- Bug in `Series.shift` and `DataFrame.shift` not supporting categorical data ([GH9416](https://github.com/pandas-dev/pandas/issues/9416))  
- Bug in `Series.map` using categorical `Series` raises `AttributeError` ([GH10324](https://github.com/pandas-dev/pandas/issues/10324))  
- Bug in `MultiIndex.get_level_values` including `Categorical` raises `AttributeError` ([GH10460](https://github.com/pandas-dev/pandas/issues/10460))  
- Bug in `pd.get_dummies` with `sparse=True` not returning `SparseDataFrame` ([GH10531](https://github.com/pandas-dev/pandas/issues/10531))  
- Bug in `Index` subtypes (such as `PeriodIndex`) not returning their own type for `.drop` and `.insert` methods ([GH10620](https://github.com/pandas-dev/pandas/issues/10620))  
- Bug in `algos.outer_join_indexer` when `right` array is empty ([GH10618](https://github.com/pandas-dev/pandas/issues/10618))  
- Bug in `filter` (regression from 0.16.0) and `transform` when grouping on multiple keys, one of which is datetime-like ([GH10114](https://github.com/pandas-dev/pandas/issues/10114))  
- Bug in `to_datetime` and `to_timedelta` causing `Index` name to be lost ([GH10875](https://github.com/pandas-dev/pandas/issues/10875))  
- Bug in `len(DataFrame.groupby)` causing `IndexError` when there’s a column containing only NaNs ([GH11016](https://github.com/pandas-dev/pandas/issues/11016))  
- Bug that caused segfault when resampling an empty Series ([GH10228](https://github.com/pandas-dev/pandas/issues/10228))  
- Bug in `DatetimeIndex` and `PeriodIndex.value_counts` resets name from its result, but retains in result’s `Index`. ([GH10150](https://github.com/pandas-dev/pandas/issues/10150))  
- Bug in `pd.eval` using `numexpr` engine coerces 1 element numpy array to scalar ([GH10546](https://github.com/pandas-dev/pandas/issues/10546))  
- Bug in `pd.concat` with `axis=0` when column is of dtype `category` ([GH10177](https://github.com/pandas-dev/pandas/issues/10177))  
- Bug in `read_msgpack` where input type is not always checked ([GH10369](https://github.com/pandas-dev/pandas/issues/10369), [GH10630](https://github.com/pandas-dev/pandas/issues/10630))  
- Bug in `pd.read_csv` with kwargs `index_col=False`, `index_col=['a', 'b']` or `dtype`
  ([GH10413](https://github.com/pandas-dev/pandas/issues/10413), [GH10467](https://github.com/pandas-dev/pandas/issues/10467), [GH10577](https://github.com/pandas-dev/pandas/issues/10577))  
- Bug in `Series.from_csv` with `header` kwarg not setting the `Series.name` or the `Series.index.name` ([GH10483](https://github.com/pandas-dev/pandas/issues/10483))  
- Bug in `groupby.var` which caused variance to be inaccurate for small float values ([GH10448](https://github.com/pandas-dev/pandas/issues/10448))  
- Bug in `Series.plot(kind='hist')` Y Label not informative ([GH10485](https://github.com/pandas-dev/pandas/issues/10485))  
- Bug in `read_csv` when using a converter which generates a `uint8` type ([GH9266](https://github.com/pandas-dev/pandas/issues/9266))  
- Bug causes memory leak in time-series line and area plot ([GH9003](https://github.com/pandas-dev/pandas/issues/9003))  
- Bug when setting a `Panel` sliced along the major or minor axes when the right-hand side is a `DataFrame` ([GH11014](https://github.com/pandas-dev/pandas/issues/11014))  
- Bug that returns `None` and does not raise `NotImplementedError` when operator functions (e.g. `.add`) of `Panel` are not implemented ([GH7692](https://github.com/pandas-dev/pandas/issues/7692))  
- Bug in line and kde plot cannot accept multiple colors when `subplots=True` ([GH9894](https://github.com/pandas-dev/pandas/issues/9894))  
- Bug in `DataFrame.plot` raises `ValueError` when color name is specified by multiple characters ([GH10387](https://github.com/pandas-dev/pandas/issues/10387))  
- Bug in left and right `align` of `Series` with `MultiIndex` may be inverted ([GH10665](https://github.com/pandas-dev/pandas/issues/10665))  
- Bug in left and right `join` of with `MultiIndex` may be inverted ([GH10741](https://github.com/pandas-dev/pandas/issues/10741))  
- Bug in `read_stata` when reading a file with a different order set in `columns` ([GH10757](https://github.com/pandas-dev/pandas/issues/10757))  
- Bug in `Categorical` may not representing properly when category contains `tz` or `Period` ([GH10713](https://github.com/pandas-dev/pandas/issues/10713))  
- Bug in `Categorical.__iter__` may not returning correct `datetime` and `Period` ([GH10713](https://github.com/pandas-dev/pandas/issues/10713))  
- Bug in indexing with a `PeriodIndex` on an object with a `PeriodIndex` ([GH4125](https://github.com/pandas-dev/pandas/issues/4125))  
- Bug in `read_csv` with `engine='c'`: EOF preceded by a comment, blank line, etc. was not handled correctly ([GH10728](https://github.com/pandas-dev/pandas/issues/10728), [GH10548](https://github.com/pandas-dev/pandas/issues/10548))  
- Reading “famafrench” data via `DataReader` results in HTTP 404 error because of the website url is changed ([GH10591](https://github.com/pandas-dev/pandas/issues/10591)).  
- Bug in `read_msgpack` where DataFrame to decode has duplicate column names ([GH9618](https://github.com/pandas-dev/pandas/issues/9618))  
- Bug in `io.common.get_filepath_or_buffer` which caused reading of valid S3 files to fail if the bucket also contained keys for which the user does not have read permission ([GH10604](https://github.com/pandas-dev/pandas/issues/10604))  
- Bug in vectorised setting of timestamp columns with python `datetime.date` and numpy `datetime64` ([GH10408](https://github.com/pandas-dev/pandas/issues/10408), [GH10412](https://github.com/pandas-dev/pandas/issues/10412))  
- Bug in `Index.take` may add unnecessary `freq` attribute ([GH10791](https://github.com/pandas-dev/pandas/issues/10791))  
- Bug in `merge` with empty `DataFrame` may raise `IndexError` ([GH10824](https://github.com/pandas-dev/pandas/issues/10824))  
- Bug in `to_latex` where unexpected keyword argument for some documented arguments ([GH10888](https://github.com/pandas-dev/pandas/issues/10888))  
- Bug in indexing of large `DataFrame` where `IndexError` is uncaught ([GH10645](https://github.com/pandas-dev/pandas/issues/10645) and [GH10692](https://github.com/pandas-dev/pandas/issues/10692))  
- Bug in `read_csv` when using the `nrows` or `chunksize` parameters if file contains only a header line ([GH9535](https://github.com/pandas-dev/pandas/issues/9535))  
- Bug in serialization of `category` types in HDF5 in presence of alternate encodings. ([GH10366](https://github.com/pandas-dev/pandas/issues/10366))  
- Bug in `pd.DataFrame` when constructing an empty DataFrame with a string dtype ([GH9428](https://github.com/pandas-dev/pandas/issues/9428))  
- Bug in `pd.DataFrame.diff` when DataFrame is not consolidated ([GH10907](https://github.com/pandas-dev/pandas/issues/10907))  
- Bug in `pd.unique` for arrays with the `datetime64` or `timedelta64` dtype that meant an array with object dtype was returned instead the original dtype ([GH9431](https://github.com/pandas-dev/pandas/issues/9431))  
- Bug in `Timedelta` raising error when slicing from 0s ([GH10583](https://github.com/pandas-dev/pandas/issues/10583))  
- Bug in `DatetimeIndex.take` and `TimedeltaIndex.take` may not raise `IndexError` against invalid index ([GH10295](https://github.com/pandas-dev/pandas/issues/10295))  
- Bug in `Series([np.nan]).astype('M8[ms]')`, which now returns `Series([pd.NaT])` ([GH10747](https://github.com/pandas-dev/pandas/issues/10747))  
- Bug in `PeriodIndex.order` reset freq ([GH10295](https://github.com/pandas-dev/pandas/issues/10295))  
- Bug in `date_range` when `freq` divides `end` as nanos ([GH10885](https://github.com/pandas-dev/pandas/issues/10885))  
- Bug in `iloc` allowing memory outside bounds of a Series to be accessed with negative integers ([GH10779](https://github.com/pandas-dev/pandas/issues/10779))  
- Bug in `read_msgpack` where encoding is not respected ([GH10581](https://github.com/pandas-dev/pandas/issues/10581))  
- Bug preventing access to the first index when using `iloc` with a list containing the appropriate negative integer ([GH10547](https://github.com/pandas-dev/pandas/issues/10547), [GH10779](https://github.com/pandas-dev/pandas/issues/10779))  
- Bug in `TimedeltaIndex` formatter causing error while trying to save `DataFrame` with `TimedeltaIndex` using `to_csv` ([GH10833](https://github.com/pandas-dev/pandas/issues/10833))  
- Bug in `DataFrame.where` when handling Series slicing ([GH10218](https://github.com/pandas-dev/pandas/issues/10218), [GH9558](https://github.com/pandas-dev/pandas/issues/9558))  
- Bug where `pd.read_gbq` throws `ValueError` when Bigquery returns zero rows ([GH10273](https://github.com/pandas-dev/pandas/issues/10273))  
- Bug in `to_json` which was causing segmentation fault when serializing 0-rank ndarray ([GH9576](https://github.com/pandas-dev/pandas/issues/9576))  
- Bug in plotting functions may raise `IndexError` when plotted on `GridSpec` ([GH10819](https://github.com/pandas-dev/pandas/issues/10819))  
- Bug in plot result may show unnecessary minor ticklabels ([GH10657](https://github.com/pandas-dev/pandas/issues/10657))  
- Bug in `groupby` incorrect computation for aggregation on `DataFrame` with `NaT` (E.g `first`, `last`, `min`). ([GH10590](https://github.com/pandas-dev/pandas/issues/10590), [GH11010](https://github.com/pandas-dev/pandas/issues/11010))  
- Bug when constructing `DataFrame` where passing a dictionary with only scalar values and specifying columns did not raise an error ([GH10856](https://github.com/pandas-dev/pandas/issues/10856))  
- Bug in `.var()` causing roundoff errors for highly similar values ([GH10242](https://github.com/pandas-dev/pandas/issues/10242))  
- Bug in `DataFrame.plot(subplots=True)` with duplicated columns outputs incorrect result ([GH10962](https://github.com/pandas-dev/pandas/issues/10962))  
- Bug in `Index` arithmetic may result in incorrect class ([GH10638](https://github.com/pandas-dev/pandas/issues/10638))  
- Bug in `date_range` results in empty if freq is negative annually, quarterly and monthly ([GH11018](https://github.com/pandas-dev/pandas/issues/11018))  
- Bug in `DatetimeIndex` cannot infer negative freq ([GH11018](https://github.com/pandas-dev/pandas/issues/11018))  
- Remove use of some deprecated numpy comparison operations, mainly in tests. ([GH10569](https://github.com/pandas-dev/pandas/issues/10569))  
- Bug in `Index` dtype may not applied properly ([GH11017](https://github.com/pandas-dev/pandas/issues/11017))  
- Bug in `io.gbq` when testing for minimum google api client version ([GH10652](https://github.com/pandas-dev/pandas/issues/10652))  
- Bug in `DataFrame` construction from nested `dict` with `timedelta` keys ([GH11129](https://github.com/pandas-dev/pandas/issues/11129))  
- Bug in `.fillna` against may raise `TypeError` when data contains datetime dtype ([GH7095](https://github.com/pandas-dev/pandas/issues/7095), [GH11153](https://github.com/pandas-dev/pandas/issues/11153))  
- Bug in `.groupby` when number of keys to group by is same as length of index ([GH11185](https://github.com/pandas-dev/pandas/issues/11185))  
- Bug in `convert_objects` where converted values might not be returned if all null and `coerce` ([GH9589](https://github.com/pandas-dev/pandas/issues/9589))  
- Bug in `convert_objects` where `copy` keyword was not respected ([GH9589](https://github.com/pandas-dev/pandas/issues/9589))  



<a id='whatsnew-0-17-0-contributors'></a>

## Contributors

A total of 112 people contributed patches to this release.  People with a
“+” by their names contributed a patch for the first time.


- Alex Rothberg  
- Andrea Bedini +  
- Andrew Rosenfeld  
- Andy Hayden  
- Andy Li +  
- Anthonios Partheniou +  
- Artemy Kolchinsky  
- Bernard Willers  
- Charlie Clark +  
- Chris +  
- Chris Whelan  
- Christoph Gohlke +  
- Christopher Whelan  
- Clark Fitzgerald  
- Clearfield Christopher +  
- Dan Ringwalt +  
- Daniel Ni +  
- Data & Code Expert Experimenting with Code on Data +  
- David Cottrell  
- David John Gagne +  
- David Kelly +  
- ETF +  
- Eduardo Schettino +  
- Egor +  
- Egor Panfilov +  
- Evan Wright  
- Frank Pinter +  
- Gabriel Araujo +  
- Garrett-R  
- Gianluca Rossi +  
- Guillaume Gay  
- Guillaume Poulin  
- Harsh Nisar +  
- Ian Henriksen +  
- Ian Hoegen +  
- Jaidev Deshpande +  
- Jan Rudolph +  
- Jan Schulz  
- Jason Swails +  
- Jeff Reback  
- Jonas Buyl +  
- Joris Van den Bossche  
- Joris Vankerschaver +  
- Josh Levy-Kramer +  
- Julien Danjou  
- Ka Wo Chen  
- Karrie Kehoe +  
- Kelsey Jordahl  
- Kerby Shedden  
- Kevin Sheppard  
- Lars Buitinck  
- Leif Johnson +  
- Luis Ortiz +  
- Mac +  
- Matt Gambogi +  
- Matt Savoie +  
- Matthew Gilbert +  
- Maximilian Roos +  
- Michelangelo D’Agostino +  
- Mortada Mehyar  
- Nick Eubank  
- Nipun Batra  
- Ondřej Čertík  
- Phillip Cloud  
- Pratap Vardhan +  
- Rafal Skolasinski +  
- Richard Lewis +  
- Rinoc Johnson +  
- Rob Levy  
- Robert Gieseke  
- Safia Abdalla +  
- Samuel Denny +  
- Saumitra Shahapure +  
- Sebastian Pölsterl +  
- Sebastian Rubbert +  
- Sheppard, Kevin +  
- Sinhrks  
- Siu Kwan Lam +  
- Skipper Seabold  
- Spencer Carrucciu +  
- Stephan Hoyer  
- Stephen Hoover +  
- Stephen Pascoe +  
- Terry Santegoeds +  
- Thomas Grainger  
- Tjerk Santegoeds +  
- Tom Augspurger  
- Vincent Davis +  
- Winterflower +  
- Yaroslav Halchenko  
- Yuan Tang (Terry) +  
- agijsberts  
- ajcr +  
- behzad nouri  
- cel4  
- chris-b1 +  
- cyrusmaher +  
- davidovitch +  
- ganego +  
- jreback  
- juricast +  
- larvian +  
- maximilianr +  
- msund +  
- rekcahpassyla  
- robertzk +  
- scls19fr  
- seth-p  
- sinhrks  
- springcoil +  
- terrytangyuan +  
- tzinckgraf +  